In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import keras
from keras.engine.topology import Layer

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [3]:
def mape_loss_func(preds, labels):
    mask = labels > 5
    return np.mean(np.fabs(labels[mask]-preds[mask])/labels[mask])

def smape_loss_func(preds, labels):
    mask= labels > 5
    return np.mean(2*np.fabs(labels[mask]-preds[mask])/(np.fabs(labels[mask])+np.fabs(preds[mask])))

def mae_loss_func(preds, labels):
    mask= labels > 5
    return np.fabs((labels[mask]-preds[mask])).mean()

def eliminate_nan(b):
    a = np.array(b)
    c = a[~np.isnan(a)]
    return c

In [27]:
#载入最近路段数据，流量数据
# distance = np.array(pd.read_csv('video_link_dis.csv',header = None))
near_road = np.array(pd.read_csv('../data/network/2small_network_nearest_road_id.csv',header = 0))
flow = np.array(pd.read_csv("../data/network/2small_network_speed.csv", header= 0)) #注意header=0 or None

In [28]:
# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式

k = 5 # 参数k为需考虑的最近路段数
t_p = 29 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(8min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(8min颗粒度) 颗粒度？
num_links = 30 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(19, t_p):  # 10 days in total
    for j in range(180-t_input-t_pre):
        image3.append(image2[:, i*180+j:i*180+j+t_input, :][:])
        label.append(flow[:, i*180+j+t_input:i*180+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*180+j+t_input:(i-1)*180+j+t_input+t_pre][:])

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前80%数据为训练集，最后20%数据为测试集
prop = 0.3
image_train = image3[:int(np.shape(image3)[0]*prop)]
image_test = image3[int(np.shape(image3)[0]*prop):]
label_train = label[:int(np.shape(label)[0]*prop)]
label_test = label[int(np.shape(label)[0]*prop):]

day_train = day[:int(np.shape(day)[0]*prop)]
day_test = day[int(np.shape(day)[0]*prop):]

(1650, 5, 12, 30)
(1650, 30, 3)
(1650, 30, 3)


In [29]:
# define merge layer
class Merge_Layer(Layer):
    def __init__(self, **kwargs):
        super(Merge_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.para1 = self.add_weight(shape=(input_shape[0][1], input_shape[0][2]),
                                     initializer='uniform', trainable=True,
                                     name='para1')
        self.para2 = self.add_weight(shape=(input_shape[1][1], input_shape[1][2]),
                                     initializer='uniform', trainable=True,
                                     name='para2')
        super(Merge_Layer, self).build(input_shape)

    def call(self, inputs):
        mat1 = inputs[0]
        mat2 = inputs[1]
        output = mat1 * self.para1 + mat2 * self.para2
        # output = mat1 * 0.1 + mat2 * 0.9
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [33]:
input_data = keras.Input(shape=(k,t_input,num_links), name='input_data')
input_HA = keras.Input(shape=(num_links, t_pre), name='input_HA')

x = keras.layers.BatchNormalization(input_shape =(k,t_input,num_links))(input_data)

x = keras.layers.Conv2D(
                           filters = 30,
                           kernel_size = 3,
                           strides = 1,
                           padding="SAME",
                           activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)

x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(
                       filters = 30,
                       kernel_size = 3,
                       strides = 1,
                       padding="SAME",
                       activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)

x = keras.layers.Dense(num_links*2*t_pre, activation='relu', name='dense_1')(x)
x = keras.layers.Dense(num_links*t_pre, activation='relu', name='dense_2')(x)

output = keras.layers.Reshape((num_links,t_pre))(x)

output_final = Merge_Layer()([output, input_HA])

# construct model
finish_model = keras.models.Model([input_data,input_HA], [output_final])

finish_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 5, 12, 30)    0                                            
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 5, 12, 30)    120         input_data[0][0]                 
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 5, 12, 30)    8130        batch_normalization_10[0][0]     
__________________________________________________________________________________________________
average_pooling2d_7 (AveragePoo (None, 5, 12, 30)    0           conv2d_7[0][0]                   
__________________________________________________________________________________________________
batch_norm

In [34]:
finish_model.compile(optimizer='adam', loss='mean_squared_error')

In [35]:
X_train = image_train
X_HA_train = day_train
label_train = label_train

In [36]:
#模型拟合与评估
finish_model.fit([X_train,X_HA_train], label_train, epochs=100, batch_size=128,
validation_data=([image_test,day_test], label_test))
# finish_model.evaluate(image_test, label_test)

Train on 495 samples, validate on 1155 samples
Epoch 1/100
495/495 [==============================] - 1s 2ms/step - loss: 403.3824 - val_loss: 402.4990
Epoch 2/100
495/495 [==============================] - 0s 187us/step - loss: 399.2983 - val_loss: 396.5200
Epoch 3/100
495/495 [==============================] - 0s 159us/step - loss: 393.7428 - val_loss: 385.8540
Epoch 4/100
495/495 [==============================] - 0s 163us/step - loss: 385.3124 - val_loss: 370.9505
Epoch 5/100
495/495 [==============================] - 0s 155us/step - loss: 372.4981 - val_loss: 351.7822
Epoch 6/100
495/495 [==============================] - 0s 147us/step - loss: 352.9308 - val_loss: 327.9426
Epoch 7/100
495/495 [==============================] - 0s 145us/step - loss: 325.2721 - val_loss: 297.3998
Epoch 8/100
495/495 [==============================] - 0s 153us/step - loss: 289.6896 - val_loss: 261.0139
Epoch 9/100
495/495 [==============================] - 0s 153us/step - loss: 250.6000 - val_loss: 2

495/495 [==============================] - 0s 137us/step - loss: 70.4170 - val_loss: 80.5790
Epoch 78/100
495/495 [==============================] - 0s 145us/step - loss: 70.9400 - val_loss: 79.7012
Epoch 79/100
495/495 [==============================] - 0s 139us/step - loss: 70.2594 - val_loss: 78.8568
Epoch 80/100
495/495 [==============================] - 0s 137us/step - loss: 69.0427 - val_loss: 78.1215
Epoch 81/100
495/495 [==============================] - 0s 139us/step - loss: 68.3727 - val_loss: 77.2977
Epoch 82/100
495/495 [==============================] - 0s 148us/step - loss: 69.5994 - val_loss: 76.4908
Epoch 83/100
495/495 [==============================] - 0s 151us/step - loss: 67.5554 - val_loss: 75.5784
Epoch 84/100
495/495 [==============================] - 0s 149us/step - loss: 66.3398 - val_loss: 75.0365
Epoch 85/100
495/495 [==============================] - 0s 145us/step - loss: 66.7422 - val_loss: 74.5193
Epoch 86/100
495/495 [==============================] - 0s 

In [37]:
#模型预测
model_pre = finish_model.predict([image_test,day_test])

In [38]:
#计算各项误差指标

mape_mean = mape_loss_func(model_pre, label_test)
smape_mean = smape_loss_func(model_pre, label_test)
mae_mean = mae_loss_func(model_pre, label_test)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.31186034260974677
smape = 0.41198701789268316
mae = 6.182956886307675


In [39]:
#模型保存
finish_model.save_weights('../model/source_base.h5')

In [40]:
#计算每条路段的误差
mape_list = []
for i in range(num_links):
    a1 = mape_loss_func(model_pre[:,i,:], label_test[:,i,:])
    mape_list.append(a1)
    print(str(i+1)+'th link')
    print(a1)

1th link
0.17099993189057397
2th link
0.14873669859536603
3th link
0.30484981053876603
4th link
0.3141808410194122
5th link
0.6268967094883933
6th link
0.15414606088704982
7th link
0.18017541519943803
8th link
0.3122541093990654
9th link
0.3232844055910491
10th link
0.48301803275022187
11th link
0.16817282156659513
12th link
0.17955317456740114
13th link
0.6566738549945366
14th link
0.3227773696383946
15th link
0.3399377729925814
16th link
0.46127914501729106
17th link
0.16213136393067387
18th link
0.1583191028116414
19th link
0.3281313902648899
20th link
0.34153200386803717
21th link
0.18488122836206536
22th link
0.4688560712587645
23th link
0.443408022934169
24th link
0.3258106831312804
25th link
0.34339281660117454
26th link
0.3132195421870117
27th link
0.17833354355498132
28th link
0.31292196926944604
29th link
0.31288050936331996
30th link
0.3350558766188119


In [41]:
mape_pd = pd.Series(mape_list)

In [42]:
mape_pd.sort_values()

1     0.148737
5     0.154146
17    0.158319
16    0.162131
10    0.168173
0     0.171000
26    0.178334
11    0.179553
6     0.180175
20    0.184881
2     0.304850
7     0.312254
28    0.312881
27    0.312922
25    0.313220
3     0.314181
13    0.322777
8     0.323284
23    0.325811
18    0.328131
29    0.335056
14    0.339938
19    0.341532
24    0.343393
22    0.443408
15    0.461279
21    0.468856
9     0.483018
4     0.626897
12    0.656674
dtype: float64